### TODO:

- ~~Create git repo~~
- Neural Network Architecture to solve image colourization problem
- Data Augmentation
- Cross-validation
- Testing a few optimizers
- Testing various loss funtions

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Conv2D, UpSampling2D, Cropping2D, InputLayer
from tensorflow.keras.models import Sequential

# Check if GPU is available and set TensorFlow to use it


class ColorizationDataGenerator(Sequence):
    def __init__(self, grayscale_dir, colorized_dir, batch_size, target_size, shuffle=True):
        self.grayscale_dir = grayscale_dir
        self.colorized_dir = colorized_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle

        self.grayscale_images = os.listdir(grayscale_dir)
        self.colorized_images = os.listdir(colorized_dir)

        # Shuffle the images if required
        if self.shuffle:
            np.random.shuffle(self.grayscale_images)

    def __len__(self):
        # Returns the number of batches per epoch
        return int(np.floor(len(self.grayscale_images) / self.batch_size))

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch
        if self.shuffle:
            np.random.shuffle(self.grayscale_images)

    def __getitem__(self, index):
        # Generate one batch of data
        batch_grayscale = self.grayscale_images[index * self.batch_size:(index + 1) * self.batch_size]
        batch_colorized = self.colorized_images[index * self.batch_size:(index + 1) * self.batch_size]

        # Load images into arrays
        grayscale_images = np.zeros((self.batch_size, *self.target_size, 1), dtype=np.float32)
        colorized_images = np.zeros((self.batch_size, *self.target_size, 3), dtype=np.float32)

        for i, grayscale_img in enumerate(batch_grayscale):
            grayscale_path = os.path.join(self.grayscale_dir, grayscale_img)
            colorized_path = os.path.join(self.colorized_dir, batch_colorized[i])

            # Load and resize grayscale image (1 channel)
            grayscale_image = load_img(grayscale_path, target_size=self.target_size, color_mode='grayscale')
            grayscale_image = img_to_array(grayscale_image) / 255.0  # Normalize to [0, 1]

            # Load and resize colorized image (3 channels)
            colorized_image = load_img(colorized_path, target_size=self.target_size, color_mode='rgb')
            colorized_image = img_to_array(colorized_image) / 255.0  # Normalize to [0, 1]

            grayscale_images[i] = grayscale_image
            colorized_images[i] = colorized_image

        return grayscale_images, colorized_images
gpus = tf.config.list_physical_devices('GPU')
if tf.config.list_physical_devices('GPU'):
    print("GPU is available and will be used for training.")
    # Define the directories
    train_grayscale_dir = 'cv_p3_images_split/train/grayscale/'
    train_colorized_dir = 'cv_p3_images_split/train/colored/'
    val_grayscale_dir = 'cv_p3_images_split/validation/grayscale/'
    val_colorized_dir = 'cv_p3_images_split/validation/colored/'

    # Create the generators
    train_generator = ColorizationDataGenerator(
        grayscale_dir=train_grayscale_dir,
        colorized_dir=train_colorized_dir,
        batch_size=32,
        target_size=(256, 256)
    )

    validation_generator = ColorizationDataGenerator(
        grayscale_dir=val_grayscale_dir,
        colorized_dir=val_colorized_dir,
        batch_size=32,
        target_size=(256, 256)
    )

    model = Sequential([
        InputLayer(input_shape=(256, 256, 1)),  # Grayscale input
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),  # Upsample to 512x512
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        Conv2D(16, (3, 3), activation='relu', padding='same'),
        Conv2D(3, (3, 3), activation='sigmoid', padding='same'),  # Output of size 256x256
        Cropping2D(cropping=((128, 128), (128, 128)))
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=50
    )

    # Save the model
    model.save('colorization_model.h5')
else:
    print("GPU is not available. Training will use the CPU.")
    


GPU is not available. Training will use the CPU.
